# Nadex Backtesting - V4: Multiple Strikes + Strategy Comparison

**Key Changes in V4:**
1. ✅ **Multiple Strikes Per Signal**: Apply ONE signal per ticker per day to ALL available strikes
2. ✅ **Strategy Comparison Framework**: Easy way to test different RSI parameters
3. ✅ **3-Tier Pricing**: $7.50 (ITM), $5.00 (ATM), $2.50 (OTM)

**Comparison:**
- V3: 690 trades (one ATM contract per signal)
- V4: ? trades (multiple strikes per signal - expect 5-10x more)

In [ ]:
%pip install pandas numpy matplotlib seaborn pyyaml boto3 --quiet

In [ ]:
import sys
sys.path.append('../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yaml
from typing import Dict
from nadex_common.utils_s3 import create_s3_clients

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

print("✓ Imports successful")

In [ ]:
# Configuration
with open('../configs/s3.yaml', 'r') as f:
    s3_cfg = yaml.safe_load(f)

clients = create_s3_clients(region=s3_cfg.get('region'))
s3_client = clients['private']
BUCKET = s3_cfg['bucket']
PREFIX = s3_cfg['prefixes']['historical']

print(f"✓ Bucket: {BUCKET}")
print(f"✓ Prefix: {PREFIX}")

## 1. Load ALL Historical Data

In [ ]:
print("Loading ALL historical data from S3...")

# FIXED: Handle S3 pagination to load ALL files (not just first 1000)
all_data = []
file_count = 0
continuation_token = None

while True:
    # List objects with pagination
    if continuation_token:
        response = s3_client.list_objects_v2(
            Bucket=BUCKET, 
            Prefix=PREFIX,
            ContinuationToken=continuation_token
        )
    else:
        response = s3_client.list_objects_v2(Bucket=BUCKET, Prefix=PREFIX)
    
    # Process files in this batch
    for obj in response.get('Contents', []):
        key = obj['Key']
        if not key.endswith('.csv'):
            continue
        try:
            obj_data = s3_client.get_object(Bucket=BUCKET, Key=key)
            df = pd.read_csv(obj_data['Body'])
            all_data.append(df)
            file_count += 1
            if file_count % 50 == 0:  # Progress indicator
                print(f"  Loaded {file_count} files...")
        except Exception as e:
            print(f"Warning: {key}: {e}")
    
    # Check if more pages exist
    if response.get('IsTruncated', False):
        continuation_token = response.get('NextContinuationToken')
    else:
        break  # No more pages

raw_data = pd.concat(all_data, ignore_index=True)
raw_data['Date'] = pd.to_datetime(raw_data['Date'], format='%d-%b-%y')

print(f"\n✓ Loaded {file_count} files")
print(f"✓ Total rows: {len(raw_data):,}")
print(f"✓ Date range: {raw_data['Date'].min().date()} to {raw_data['Date'].max().date()}")
print(f"✓ Unique tickers: {raw_data['Ticker'].nunique()}")
print(f"✓ Unique dates: {raw_data['Date'].nunique()}")

display(raw_data.head(10))

## 2. Prepare Daily Data (Keep ALL Strikes)

**Key Change:** We DON'T aggregate to one contract per day.  
Instead, we keep ALL strikes and apply the signal to each one.

In [ ]:
# Sort data for processing
data = raw_data.sort_values(['Ticker', 'Date', 'Strike Price']).reset_index(drop=True)

print(f"✓ Keeping ALL {len(data):,} contracts (not aggregating)")
print(f"✓ Will generate ONE signal per ticker per day")
print(f"✓ That signal applies to ALL strikes for that ticker on that day")

# Show example of multiple strikes per day
sample = data[data['Ticker'] == data['Ticker'].iloc[0]].head(15)
print(f"\nExample: {sample['Ticker'].iloc[0]} on {sample['Date'].iloc[0].date()}")
print(f"Has {len(sample[sample['Date'] == sample['Date'].iloc[0]])} different strikes")
display(sample)

## 3. 3-Tier Pricing Model

In [ ]:
def calculate_tier_entry_cost(exp_value: float, strike_price: float) -> float:
    """3-Tier pricing: $7.50 (ITM), $5.00 (ATM), $2.50 (OTM)"""
    threshold = strike_price * 0.01
    diff = exp_value - strike_price
    
    if diff > threshold:
        return 7.50  # Far ITM
    elif diff < -threshold:
        return 2.50  # Far OTM
    else:
        return 5.00  # ATM

print("✓ 3-tier pricing model ready")

## 4. Strategy Implementation with Multi-Strike Support

In [ ]:
def calculate_rsi(prices: pd.Series, period: int = 14) -> pd.Series:
    """Calculate RSI indicator."""
    delta = prices.diff()
    gain = delta.where(delta > 0, 0).rolling(window=period).mean()
    loss = -delta.where(delta < 0, 0).rolling(window=period).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

def backtest_multi_strike(data: pd.DataFrame, rsi_period: int = 14,
                         oversold: float = 30, overbought: float = 70) -> pd.DataFrame:
    """
    Backtest with multiple strikes per signal.
    
    Process:
    1. For each ticker, calculate RSI on Exp Value time series
    2. Generate ONE signal per day based on RSI
    3. Apply that signal to ALL strikes for that ticker on that day
    4. Calculate P&L for each strike separately
    """
    all_results = []
    
    for ticker in data['Ticker'].unique():
        ticker_data = data[data['Ticker'] == ticker].copy()
        
        # Get daily Exp Value (use mean across strikes for that day)
        daily_exp = ticker_data.groupby('Date')['Exp Value'].mean().reset_index()
        daily_exp = daily_exp.sort_values('Date').reset_index(drop=True)
        
        # Calculate RSI on daily Exp Value
        daily_exp['rsi'] = calculate_rsi(daily_exp['Exp Value'], rsi_period)
        
        # Generate signals
        daily_exp['signal'] = 0
        daily_exp.loc[daily_exp['rsi'] < oversold, 'signal'] = 1   # BUY
        daily_exp.loc[daily_exp['rsi'] > overbought, 'signal'] = -1  # SELL
        
        # Merge signals back to ALL strikes
        ticker_data = ticker_data.merge(
            daily_exp[['Date', 'rsi', 'signal']], 
            on='Date', 
            how='left'
        )
        
        # Calculate P&L for each strike that gets a signal
        trades = ticker_data[ticker_data['signal'] != 0].copy()
        for idx in trades.index:
            row = ticker_data.loc[idx]
            entry_cost = calculate_tier_entry_cost(row['Exp Value'], row['Strike Price'])
            pnl = (10.0 - entry_cost) if row['In the Money'] == 1 else -entry_cost
            ticker_data.loc[idx, 'entry_cost'] = entry_cost
            ticker_data.loc[idx, 'pnl'] = pnl
        
        all_results.append(ticker_data)
    
    return pd.concat(all_results, ignore_index=True)

print("✓ Multi-strike backtesting function ready")

## 5. Run Baseline with Multi-Strike

In [ ]:
print("Running multi-strike backtest...")
print("Configuration: RSI(14), Oversold=30, Overbought=70")
print("Strategy: Apply ONE signal per ticker per day to ALL strikes\n")

results = backtest_multi_strike(data, rsi_period=14, oversold=30, overbought=70)
trades = results[results['signal'] != 0].copy()

print(f"✓ Backtest complete!")
print(f"✓ Total trades: {len(trades):,}")

## 6. Results Analysis

In [ ]:
wins = trades[trades['pnl'] > 0]
losses = trades[trades['pnl'] < 0]
tier_counts = trades['entry_cost'].value_counts().sort_index()

print("=" * 70)
print("📊 MULTI-STRIKE RESULTS")
print("=" * 70)
print(f"Total Trades:           {len(trades):,}")
print(f"Winning Trades:         {len(wins):,}")
print(f"Losing Trades:          {len(losses):,}")
print(f"Win Rate:               {len(wins)/len(trades):.2%}")
print(f"\nTotal P&L:              ${trades['pnl'].sum():.2f}")
print(f"Average Win:            ${wins['pnl'].mean():.2f}" if len(wins) > 0 else "N/A")
print(f"Average Loss:           ${losses['pnl'].mean():.2f}" if len(losses) > 0 else "N/A")
print(f"\nEntry Cost Distribution:")
for cost, count in tier_counts.items():
    pct = count / len(trades) * 100
    print(f"  ${cost:.2f}: {count:,} trades ({pct:.1f}%)")
print(f"\nAvg Entry Cost:         ${trades['entry_cost'].mean():.2f}")
print(f"Total Capital Used:     ${trades['entry_cost'].sum():.2f}")
print(f"Total Return:           {(trades['pnl'].sum() / trades['entry_cost'].sum() * 100):.2f}%")
print(f"\nSharpe Ratio:           {(trades['pnl'].mean() / trades['pnl'].std()) * np.sqrt(252):.2f}" if len(trades) > 1 else "N/A")
print("=" * 70)

print("\nComparison to V3 (Single ATM):")
print(f"  V3 Trades: 690")
print(f"  V4 Trades: {len(trades):,}")
print(f"  Multiplier: {len(trades)/690:.1f}x")

print("\nSample Trades:")
display(trades[['Date', 'Ticker', 'Exp Value', 'Strike Price', 'rsi', 
                'entry_cost', 'In the Money', 'pnl']].head(20))

## 7. Strategy Comparison Framework

Test multiple RSI configurations to find the best parameters.

In [ ]:
print("Running strategy comparison...")
print("Testing different RSI parameters\n")

# Define strategies to test
strategies = {
    'Baseline (14, 30/70)': {'rsi_period': 14, 'oversold': 30, 'overbought': 70},
    'Conservative (14, 25/75)': {'rsi_period': 14, 'oversold': 25, 'overbought': 75},
    'Aggressive (14, 35/65)': {'rsi_period': 14, 'oversold': 35, 'overbought': 65},
    'Fast RSI (7, 30/70)': {'rsi_period': 7, 'oversold': 30, 'overbought': 70},
    'Slow RSI (21, 30/70)': {'rsi_period': 21, 'oversold': 30, 'overbought': 70},
}

comparison_results = []

for name, params in strategies.items():
    print(f"  Testing: {name}...")
    results = backtest_multi_strike(data, **params)
    trades = results[results['signal'] != 0]
    
    wins = trades[trades['pnl'] > 0]
    
    comparison_results.append({
        'strategy': name,
        'total_trades': len(trades),
        'win_rate': len(wins) / len(trades) if len(trades) > 0 else 0,
        'total_pnl': trades['pnl'].sum(),
        'avg_pnl': trades['pnl'].mean() if len(trades) > 0 else 0,
        'total_capital': trades['entry_cost'].sum() if len(trades) > 0 else 0,
        'total_return_pct': (trades['pnl'].sum() / trades['entry_cost'].sum() * 100) if len(trades) > 0 and trades['entry_cost'].sum() > 0 else 0,
        'sharpe_ratio': (trades['pnl'].mean() / trades['pnl'].std()) * np.sqrt(252) if len(trades) > 1 and trades['pnl'].std() > 0 else 0,
    })

comparison_df = pd.DataFrame(comparison_results)

print("\n" + "=" * 100)
print("📊 STRATEGY COMPARISON")
print("=" * 100)
display(comparison_df.round(2))

print("\n🏆 BEST PERFORMERS:")
print("-" * 100)
print(f"Best by Total P&L:      {comparison_df.loc[comparison_df['total_pnl'].idxmax(), 'strategy']}")
print(f"Best by Win Rate:       {comparison_df.loc[comparison_df['win_rate'].idxmax(), 'strategy']}")
print(f"Best by Total Return:   {comparison_df.loc[comparison_df['total_return_pct'].idxmax(), 'strategy']}")
print(f"Best by Sharpe Ratio:   {comparison_df.loc[comparison_df['sharpe_ratio'].idxmax(), 'strategy']}")

## 8. Visualization: Strategy Comparison

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# Total P&L
axes[0, 0].barh(comparison_df['strategy'], comparison_df['total_pnl'])
axes[0, 0].set_xlabel('Total P&L ($)')
axes[0, 0].set_title('Total P&L by Strategy')
axes[0, 0].axvline(x=0, color='red', linestyle='--', alpha=0.5)
axes[0, 0].grid(axis='x', alpha=0.3)

# Win Rate
axes[0, 1].barh(comparison_df['strategy'], comparison_df['win_rate'] * 100)
axes[0, 1].set_xlabel('Win Rate (%)')
axes[0, 1].set_title('Win Rate by Strategy')
axes[0, 1].axvline(x=50, color='red', linestyle='--', alpha=0.5, label='Break-even')
axes[0, 1].grid(axis='x', alpha=0.3)
axes[0, 1].legend()

# Total Return %
axes[1, 0].barh(comparison_df['strategy'], comparison_df['total_return_pct'])
axes[1, 0].set_xlabel('Total Return (%)')
axes[1, 0].set_title('Return % by Strategy')
axes[1, 0].axvline(x=0, color='red', linestyle='--', alpha=0.5)
axes[1, 0].grid(axis='x', alpha=0.3)

# Sharpe Ratio
axes[1, 1].barh(comparison_df['strategy'], comparison_df['sharpe_ratio'])
axes[1, 1].set_xlabel('Sharpe Ratio')
axes[1, 1].set_title('Risk-Adjusted Returns (Sharpe)')
axes[1, 1].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

## 9. Cumulative P&L Chart

In [ ]:
# Use baseline results for cumulative P&L
trades_sorted = trades.sort_values('Date').copy()
trades_sorted['cumulative_pnl'] = trades_sorted['pnl'].cumsum()

plt.figure(figsize=(14, 6))
plt.plot(trades_sorted['Date'], trades_sorted['cumulative_pnl'], linewidth=2)
plt.axhline(y=0, color='r', linestyle='--', alpha=0.3)
plt.fill_between(trades_sorted['Date'], 0, trades_sorted['cumulative_pnl'], 
                 where=(trades_sorted['cumulative_pnl'] >= 0), alpha=0.3, color='green')
plt.fill_between(trades_sorted['Date'], 0, trades_sorted['cumulative_pnl'], 
                 where=(trades_sorted['cumulative_pnl'] < 0), alpha=0.3, color='red')
plt.xlabel('Date')
plt.ylabel('Cumulative P&L ($)')
plt.title(f'Cumulative P&L Over Time - Multi-Strike (Final: ${trades_sorted["cumulative_pnl"].iloc[-1]:.2f})')
plt.grid(alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 10. Conservative Strategy Deep-Dive

Based on the results above, the Conservative strategy (RSI 14, 25/75 thresholds) performed best.
Let's analyze it in more detail.

In [ ]:
# Run Conservative strategy
conservative_results = backtest_multi_strike(data, rsi_period=14, oversold=25, overbought=75)
conservative_trades = conservative_results[conservative_results['signal'] != 0].copy()

print("Conservative Strategy Deep Analysis")
print("=" * 70)
print(f"Total Trades: {len(conservative_trades):,}")
print(f"Win Rate: {(len(conservative_trades[conservative_trades['pnl'] > 0]) / len(conservative_trades)):.2%}")
print(f"Total P&L: ${conservative_trades['pnl'].sum():.2f}")
print(f"Avg Trade P&L: ${conservative_trades['pnl'].mean():.2f}")
print(f"Total Return: {(conservative_trades['pnl'].sum() / conservative_trades['entry_cost'].sum() * 100):.2f}%")

## 11. Ticker-Level Performance Analysis

In [ ]:
# Analyze performance by ticker
ticker_perf = trades.groupby('Ticker').agg({
    'pnl': ['count', 'sum', 'mean'],
    'In the Money': 'mean'
}).round(2)

ticker_perf.columns = ['Trade_Count', 'Total_PnL', 'Avg_PnL', 'Win_Rate']
ticker_perf = ticker_perf.sort_values('Total_PnL', ascending=False)

print("\nPerformance by Ticker:")
print("=" * 70)
display(ticker_perf)

# Visualize
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

ticker_perf['Total_PnL'].plot(kind='barh', ax=ax1)
ax1.set_xlabel('Total P&L ($)')
ax1.set_title('Total P&L by Ticker')
ax1.axvline(x=0, color='red', linestyle='--', alpha=0.5)

ticker_perf['Win_Rate'].plot(kind='barh', ax=ax2)
ax2.set_xlabel('Win Rate')
ax2.set_title('Win Rate by Ticker')
ax2.axvline(x=0.5, color='red', linestyle='--', alpha=0.5, label='Break-even')
ax2.legend()

plt.tight_layout()
plt.show()

## 12. Portfolio P&L Analysis Over Time

In [ ]:
# Create portfolio P&L chart with better granularity
trades_by_date = trades.groupby('Date')['pnl'].sum().reset_index()
trades_by_date['cumulative_pnl'] = trades_by_date['pnl'].cumsum()

plt.figure(figsize=(16, 8))

# Main cumulative P&L
plt.subplot(2, 1, 1)
plt.plot(trades_by_date['Date'], trades_by_date['cumulative_pnl'], linewidth=2, label='Cumulative P&L')
plt.fill_between(trades_by_date['Date'], 0, trades_by_date['cumulative_pnl'],
                 where=(trades_by_date['cumulative_pnl'] >= 0), alpha=0.3, color='green')
plt.fill_between(trades_by_date['Date'], 0, trades_by_date['cumulative_pnl'],
                 where=(trades_by_date['cumulative_pnl'] < 0), alpha=0.3, color='red')
plt.axhline(y=0, color='black', linestyle='-', alpha=0.3)
plt.ylabel('Cumulative P&L ($)')
plt.title(f'Portfolio Cumulative P&L (Final: ${trades_by_date["cumulative_pnl"].iloc[-1]:.2f})')
plt.grid(alpha=0.3)
plt.legend()

# Daily P&L
plt.subplot(2, 1, 2)
colors = ['green' if x > 0 else 'red' for x in trades_by_date['pnl']]
plt.bar(trades_by_date['Date'], trades_by_date['pnl'], color=colors, alpha=0.6)
plt.axhline(y=0, color='black', linestyle='-', alpha=0.3)
plt.ylabel('Daily P&L ($)')
plt.xlabel('Date')
plt.title('Daily P&L')
plt.grid(alpha=0.3)
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

# Summary statistics
print("\nPortfolio Statistics:")
print("=" * 70)
print(f"Best Day: ${trades_by_date['pnl'].max():.2f}")
print(f"Worst Day: ${trades_by_date['pnl'].min():.2f}")
print(f"Avg Daily P&L: ${trades_by_date['pnl'].mean():.2f}")
print(f"Winning Days: {len(trades_by_date[trades_by_date['pnl'] > 0])}")
print(f"Losing Days: {len(trades_by_date[trades_by_date['pnl'] < 0])}")
print(f"Win Day %: {(len(trades_by_date[trades_by_date['pnl'] > 0]) / len(trades_by_date)):.2%}")

## 13. Exclude Bottom Performers & Re-test Conservative Strategy

**Purpose:** Improve results by excluding the worst performing tickers.

**Process:**
1. Identify the bottom 5 tickers by total P&L from baseline results
2. Exclude them from the dataset
3. Re-run the Conservative strategy (RSI 14, 25/75) on the filtered data
4. Compare results

In [ ]:
# Step 1: Identify bottom 5 tickers from baseline results
print("Analyzing ticker performance from baseline results...\n")

# Ensure we have the baseline results
if 'trades' not in locals() or trades is None or len(trades) == 0:
    print("ERROR: No baseline trades found. Please run Section 5 first!")
else:
    ticker_pnl = trades.groupby('Ticker')['pnl'].agg(['sum', 'count']).sort_values('sum')
    
    print("All Tickers Ranked by Total P&L:")
    print("=" * 70)
    display(ticker_pnl)
    
    # Get bottom 5
    bottom_5 = ticker_pnl.head(5).index.tolist()
    
    print(f"\n🔻 BOTTOM 5 TICKERS TO EXCLUDE:")
    for i, ticker in enumerate(bottom_5, 1):
        pnl = ticker_pnl.loc[ticker, 'sum']
        count = ticker_pnl.loc[ticker, 'count']
        print(f"  {i}. {ticker}: ${pnl:.2f} ({count} trades)")
    
    # Create filtered dataset (all tickers EXCEPT bottom 5)
    # Use 'data' from section 2 which has ALL the original data
    all_tickers = data['Ticker'].unique()
    filtered_tickers = [t for t in all_tickers if t not in bottom_5]
    
    print(f"\n✅ KEEPING {len(filtered_tickers)} TICKERS (excluding bottom 5)")
    print(f"Excluded: {bottom_5}")
    print(f"Keeping: {sorted(filtered_tickers)}")
    
    filtered_data = data[data['Ticker'].isin(filtered_tickers)].copy()
    
    print(f"\nFiltered Data Stats:")
    print(f"  Original contracts: {len(data):,}")
    print(f"  Filtered contracts: {len(filtered_data):,}")
    print(f"  Reduction: {(1 - len(filtered_data)/len(data)):.1%}")
    print(f"  Original tickers: {data['Ticker'].nunique()}")
    print(f"  Filtered tickers: {filtered_data['Ticker'].nunique()}")
    print(f"  Date range: {filtered_data['Date'].min().date()} to {filtered_data['Date'].max().date()}")

In [ ]:
# Step 2: Run Conservative strategy (RSI 14, 25/75) on filtered data
print("Running CONSERVATIVE strategy (RSI 14, 25/75) on filtered data...\n")

if len(filtered_data) == 0:
    print("ERROR: Filtered data is empty!")
else:
    filtered_results = backtest_multi_strike(filtered_data, rsi_period=14, oversold=25, overbought=75)
    filtered_trades = filtered_results[filtered_results['signal'] != 0].copy()
    
    if len(filtered_trades) == 0:
        print("WARNING: No trades generated with this strategy!")
    else:
        wins_f = filtered_trades[filtered_trades['pnl'] > 0]
        losses_f = filtered_trades[filtered_trades['pnl'] < 0]
        
        print("=" * 70)
        print("📊 FILTERED RESULTS (Conservative Strategy, Bottom 5 Excluded)")
        print("=" * 70)
        print(f"Total Trades:           {len(filtered_trades):,}")
        print(f"Winning Trades:         {len(wins_f):,}")
        print(f"Losing Trades:          {len(losses_f):,}")
        print(f"Win Rate:               {len(wins_f)/len(filtered_trades):.2%}")
        print(f"\nTotal P&L:              ${filtered_trades['pnl'].sum():.2f}")
        print(f"Average Win:            ${wins_f['pnl'].mean():.2f}" if len(wins_f) > 0 else "N/A")
        print(f"Average Loss:           ${losses_f['pnl'].mean():.2f}" if len(losses_f) > 0 else "N/A")
        print(f"Avg Trade P&L:          ${filtered_trades['pnl'].mean():.2f}")
        print(f"\nTotal Capital Used:     ${filtered_trades['entry_cost'].sum():.2f}")
        print(f"Total Return:           {(filtered_trades['pnl'].sum() / filtered_trades['entry_cost'].sum() * 100):.2f}%")
        print(f"Sharpe Ratio:           {(filtered_trades['pnl'].mean() / filtered_trades['pnl'].std()) * np.sqrt(252):.2f}" if len(filtered_trades) > 1 and filtered_trades['pnl'].std() > 0 else "N/A")
        print("=" * 70)

In [ ]:
# Step 3: Compare to original Conservative results
if len(filtered_trades) > 0:
    print("\n" + "=" * 70)
    print("📊 COMPARISON: Before vs After Excluding Bottom 5")
    print("=" * 70)
    print(f"Strategy: Conservative (RSI 14, 25/75)\n")
    
    # Get original conservative results (re-run on full data)
    print("Re-running Conservative strategy on FULL data for comparison...")
    original_cons_results = backtest_multi_strike(data, rsi_period=14, oversold=25, overbought=75)
    original_cons_trades = original_cons_results[original_cons_results['signal'] != 0].copy()
    
    print(f"\nBEFORE (All {data['Ticker'].nunique()} Tickers):")
    print(f"  Trades: {len(original_cons_trades):,}")
    print(f"  Win Rate: {(len(original_cons_trades[original_cons_trades['pnl'] > 0]) / len(original_cons_trades)):.2%}")
    print(f"  Total P&L: ${original_cons_trades['pnl'].sum():.2f}")
    print(f"  Total Return: {(original_cons_trades['pnl'].sum() / original_cons_trades['entry_cost'].sum() * 100):.2f}%")
    
    print(f"\nAFTER ({filtered_data['Ticker'].nunique()} Tickers - Bottom 5 Excluded):")
    print(f"  Trades: {len(filtered_trades):,}")
    print(f"  Win Rate: {len(wins_f)/len(filtered_trades):.2%}")
    print(f"  Total P&L: ${filtered_trades['pnl'].sum():.2f}")
    print(f"  Total Return: {(filtered_trades['pnl'].sum() / filtered_trades['entry_cost'].sum() * 100):.2f}%")
    
    print(f"\n📈 IMPROVEMENT:")
    pnl_improvement = filtered_trades['pnl'].sum() - original_cons_trades['pnl'].sum()
    orig_return = original_cons_trades['pnl'].sum() / original_cons_trades['entry_cost'].sum()
    filt_return = filtered_trades['pnl'].sum() / filtered_trades['entry_cost'].sum()
    return_improvement = filt_return - orig_return
    
    print(f"  P&L Change: ${pnl_improvement:.2f} ({pnl_improvement/original_cons_trades['pnl'].sum()*100:+.1f}%)")
    print(f"  Return Change: {return_improvement * 100:+.2f} percentage points")
    print(f"  Trade Reduction: {len(original_cons_trades) - len(filtered_trades):,} trades ({(1-len(filtered_trades)/len(original_cons_trades))*100:.1f}% fewer)")
    print("=" * 70)
    
    # Store for visualization
    comparison_data = {
        'original_trades': original_cons_trades,
        'filtered_trades': filtered_trades
    }

In [ ]:
# Step 4: Visualize Comparison
if len(filtered_trades) > 0:
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Original cumulative P&L
    orig_sorted = comparison_data['original_trades'].sort_values('Date').copy()
    orig_sorted['cumulative_pnl'] = orig_sorted['pnl'].cumsum()
    
    axes[0].plot(orig_sorted['Date'], orig_sorted['cumulative_pnl'], linewidth=2, color='red', label='All Tickers')
    axes[0].fill_between(orig_sorted['Date'], 0, orig_sorted['cumulative_pnl'],
                         where=(orig_sorted['cumulative_pnl'] >= 0), alpha=0.3, color='green')
    axes[0].fill_between(orig_sorted['Date'], 0, orig_sorted['cumulative_pnl'],
                         where=(orig_sorted['cumulative_pnl'] < 0), alpha=0.3, color='red')
    axes[0].axhline(y=0, color='black', linestyle='--', alpha=0.3)
    axes[0].set_xlabel('Date')
    axes[0].set_ylabel('Cumulative P&L ($)')
    axes[0].set_title(f'BEFORE: All Tickers\nFinal: ${orig_sorted["cumulative_pnl"].iloc[-1]:.2f}')
    axes[0].grid(alpha=0.3)
    axes[0].tick_params(axis='x', rotation=45)
    axes[0].legend()
    
    # Filtered cumulative P&L
    filt_sorted = comparison_data['filtered_trades'].sort_values('Date').copy()
    filt_sorted['cumulative_pnl'] = filt_sorted['pnl'].cumsum()
    
    axes[1].plot(filt_sorted['Date'], filt_sorted['cumulative_pnl'], linewidth=2, color='blue', label='Bottom 5 Excluded')
    axes[1].fill_between(filt_sorted['Date'], 0, filt_sorted['cumulative_pnl'],
                         where=(filt_sorted['cumulative_pnl'] >= 0), alpha=0.3, color='green')
    axes[1].fill_between(filt_sorted['Date'], 0, filt_sorted['cumulative_pnl'],
                         where=(filt_sorted['cumulative_pnl'] < 0), alpha=0.3, color='red')
    axes[1].axhline(y=0, color='black', linestyle='--', alpha=0.3)
    axes[1].set_xlabel('Date')
    axes[1].set_ylabel('Cumulative P&L ($)')
    axes[1].set_title(f'AFTER: Bottom 5 Excluded\nFinal: ${filt_sorted["cumulative_pnl"].iloc[-1]:.2f}')
    axes[1].grid(alpha=0.3)
    axes[1].tick_params(axis='x', rotation=45)
    axes[1].legend()
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n✅ Improvement: ${filt_sorted['cumulative_pnl'].iloc[-1] - orig_sorted['cumulative_pnl'].iloc[-1]:.2f}")

## 14. Strike Range Filtering Analysis

**Purpose:** Test if filtering to nearby strikes (±5%) improves strategy performance.

**Hypothesis:** Trading only strikes within ±5% of Expected Value will:
- Reduce noise from extreme OTM/ITM strikes
- Improve capital efficiency
- Potentially increase win rate

**Comparison:**
1. Conservative (RSI 14, 25/75) - ALL strikes (baseline)
2. Conservative + ±5% filter
3. Conservative + ±5% filter + Bottom 5 tickers excluded

In [ ]:
# Step 1: Create ±5% filtered dataset
print("Creating ±5% strike filter...\n")

# Calculate distance from Exp Value as percentage
data['distance_pct'] = abs(data['Strike Price'] - data['Exp Value']) / data['Exp Value']

# Filter to ±5%
filtered_5pct = data[data['distance_pct'] <= 0.05].copy()

print("📊 Strike Filtering Results:")
print("=" * 70)
print(f"Original data: {len(data):,} contracts")
print(f"Filtered (±5%): {len(filtered_5pct):,} contracts")
print(f"Reduction: {(1 - len(filtered_5pct)/len(data)):.1%}")
print(f"\nOriginal tickers: {data['Ticker'].nunique()}")
print(f"Filtered tickers: {filtered_5pct['Ticker'].nunique()}")
print(f"Date range: {filtered_5pct['Date'].min().date()} to {filtered_5pct['Date'].max().date()}")
print("=" * 70)

In [ ]:
# Step 2: Run Conservative strategy on filtered data
print("\nRunning Conservative strategy (RSI 14, 25/75) on ±5% filtered data...\n")

filtered_results = backtest_multi_strike(filtered_5pct, rsi_period=14, oversold=25, overbought=75)
filtered_trades = filtered_results[filtered_results['signal'] != 0].copy()

if len(filtered_trades) == 0:
    print("WARNING: No trades generated!")
else:
    wins_filt = filtered_trades[filtered_trades['pnl'] > 0]
    losses_filt = filtered_trades[filtered_trades['pnl'] < 0]
    
    print("=" * 70)
    print("📊 FILTERED RESULTS (Conservative + ±5% Strike Filter)")
    print("=" * 70)
    print(f"Total Trades:           {len(filtered_trades):,}")
    print(f"Winning Trades:         {len(wins_filt):,}")
    print(f"Losing Trades:          {len(losses_filt):,}")
    print(f"Win Rate:               {len(wins_filt)/len(filtered_trades):.2%}")
    print(f"\nTotal P&L:              ${filtered_trades['pnl'].sum():.2f}")
    print(f"Average Win:            ${wins_filt['pnl'].mean():.2f}" if len(wins_filt) > 0 else "N/A")
    print(f"Average Loss:           ${losses_filt['pnl'].mean():.2f}" if len(losses_filt) > 0 else "N/A")
    print(f"Avg Trade P&L:          ${filtered_trades['pnl'].mean():.2f}")
    print(f"\nTotal Capital Used:     ${filtered_trades['entry_cost'].sum():.2f}")
    print(f"Total Return:           {(filtered_trades['pnl'].sum() / filtered_trades['entry_cost'].sum() * 100):.2f}%")
    print(f"Sharpe Ratio:           {(filtered_trades['pnl'].mean() / filtered_trades['pnl'].std()) * np.sqrt(252):.2f}" if len(filtered_trades) > 1 and filtered_trades['pnl'].std() > 0 else "N/A")
    print("=" * 70)

In [ ]:
# Step 3: Filter BOTH by ±5% AND exclude bottom 5 tickers
if 'bottom_5' in locals() and 'filtered_data' in locals():
    print("\nApplying BOTH ±5% filter AND bottom 5 exclusion...\n")
    
    # Start with filtered_data from Section 13 (bottom 5 excluded)
    double_filtered = filtered_data[filtered_data['Ticker'].isin(filtered_tickers)].copy()
    
    # Calculate distance and apply ±5% filter
    double_filtered['distance_pct'] = abs(double_filtered['Strike Price'] - double_filtered['Exp Value']) / double_filtered['Exp Value']
    double_filtered = double_filtered[double_filtered['distance_pct'] <= 0.05].copy()
    
    print(f"Original data: {len(data):,} contracts")
    print(f"Bottom 5 excluded: {len(filtered_data):,} contracts")
    print(f"Both filters applied: {len(double_filtered):,} contracts")
    print(f"Total reduction: {(1 - len(double_filtered)/len(data)):.1%}\n")
    
    # Run backtest
    double_results = backtest_multi_strike(double_filtered, rsi_period=14, oversold=25, overbought=75)
    double_trades = double_results[double_results['signal'] != 0].copy()
    
    if len(double_trades) > 0:
        wins_dbl = double_trades[double_trades['pnl'] > 0]
        losses_dbl = double_trades[double_trades['pnl'] < 0]
        
        print("=" * 70)
        print("📊 DOUBLE FILTERED (±5% + Bottom 5 Excluded)")
        print("=" * 70)
        print(f"Total Trades:           {len(double_trades):,}")
        print(f"Winning Trades:         {len(wins_dbl):,}")
        print(f"Losing Trades:          {len(losses_dbl):,}")
        print(f"Win Rate:               {len(wins_dbl)/len(double_trades):.2%}")
        print(f"\nTotal P&L:              ${double_trades['pnl'].sum():.2f}")
        print(f"Average Win:            ${wins_dbl['pnl'].mean():.2f}" if len(wins_dbl) > 0 else "N/A")
        print(f"Average Loss:           ${losses_dbl['pnl'].mean():.2f}" if len(losses_dbl) > 0 else "N/A")
        print(f"Avg Trade P&L:          ${double_trades['pnl'].mean():.2f}")
        print(f"\nTotal Capital Used:     ${double_trades['entry_cost'].sum():.2f}")
        print(f"Total Return:           {(double_trades['pnl'].sum() / double_trades['entry_cost'].sum() * 100):.2f}%")
        print(f"Sharpe Ratio:           {(double_trades['pnl'].mean() / double_trades['pnl'].std()) * np.sqrt(252):.2f}" if len(double_trades) > 1 and double_trades['pnl'].std() > 0 else "N/A")
        print("=" * 70)
else:
    print("\n⚠️  Note: Run Section 13 first to enable double filtering (±5% + Bottom 5 exclusion)")

In [ ]:
# Step 4: Comprehensive Comparison
print("\n" + "=" * 70)
print("📊 COMPREHENSIVE COMPARISON - Conservative Strategy (RSI 14, 25/75)")
print("=" * 70)

# Re-run baseline for comparison
baseline_results = backtest_multi_strike(data, rsi_period=14, oversold=25, overbought=75)
baseline_trades = baseline_results[baseline_results['signal'] != 0].copy()

print(f"\n1️⃣  BASELINE (All Strikes, All Tickers):")
print(f"   Trades: {len(baseline_trades):,}")
print(f"   Win Rate: {(len(baseline_trades[baseline_trades['pnl'] > 0]) / len(baseline_trades)):.2%}")
print(f"   Total P&L: ${baseline_trades['pnl'].sum():.2f}")
print(f"   Return: {(baseline_trades['pnl'].sum() / baseline_trades['entry_cost'].sum() * 100):.2f}%")
print(f"   Capital Used: ${baseline_trades['entry_cost'].sum():.2f}")

if len(filtered_trades) > 0:
    print(f"\n2️⃣  WITH ±5% STRIKE FILTER:")
    print(f"   Trades: {len(filtered_trades):,} ({(len(filtered_trades)/len(baseline_trades) - 1)*100:+.1f}%)")
    print(f"   Win Rate: {len(wins_filt)/len(filtered_trades):.2%} ({(len(wins_filt)/len(filtered_trades) - len(baseline_trades[baseline_trades['pnl'] > 0])/len(baseline_trades))*100:+.1f}pp)")
    print(f"   Total P&L: ${filtered_trades['pnl'].sum():.2f} ({(filtered_trades['pnl'].sum() - baseline_trades['pnl'].sum()):+.2f})")
    print(f"   Return: {(filtered_trades['pnl'].sum() / filtered_trades['entry_cost'].sum() * 100):.2f}% ({(filtered_trades['pnl'].sum()/filtered_trades['entry_cost'].sum() - baseline_trades['pnl'].sum()/baseline_trades['entry_cost'].sum())*100:+.2f}pp)")
    print(f"   Capital Used: ${filtered_trades['entry_cost'].sum():.2f} ({(filtered_trades['entry_cost'].sum()/baseline_trades['entry_cost'].sum() - 1)*100:+.1f}%)")

if 'double_trades' in locals() and len(double_trades) > 0:
    print(f"\n3️⃣  WITH ±5% FILTER + BOTTOM 5 EXCLUDED:")
    print(f"   Trades: {len(double_trades):,} ({(len(double_trades)/len(baseline_trades) - 1)*100:+.1f}%)")
    print(f"   Win Rate: {len(wins_dbl)/len(double_trades):.2%} ({(len(wins_dbl)/len(double_trades) - len(baseline_trades[baseline_trades['pnl'] > 0])/len(baseline_trades))*100:+.1f}pp)")
    print(f"   Total P&L: ${double_trades['pnl'].sum():.2f} ({(double_trades['pnl'].sum() - baseline_trades['pnl'].sum()):+.2f})")
    print(f"   Return: {(double_trades['pnl'].sum() / double_trades['entry_cost'].sum() * 100):.2f}% ({(double_trades['pnl'].sum()/double_trades['entry_cost'].sum() - baseline_trades['pnl'].sum()/baseline_trades['entry_cost'].sum())*100:+.2f}pp)")
    print(f"   Capital Used: ${double_trades['entry_cost'].sum():.2f} ({(double_trades['entry_cost'].sum()/baseline_trades['entry_cost'].sum() - 1)*100:+.1f}%)")

print("\n" + "=" * 70)
print("\n💡 KEY INSIGHTS:")
if len(filtered_trades) > 0:
    pnl_change = filtered_trades['pnl'].sum() - baseline_trades['pnl'].sum()
    if pnl_change > 0:
        print(f"   ✅ Strike filtering IMPROVED P&L by ${pnl_change:.2f}")
    else:
        print(f"   ❌ Strike filtering REDUCED P&L by ${abs(pnl_change):.2f}")
    
    capital_efficiency = (filtered_trades['pnl'].sum() / filtered_trades['entry_cost'].sum()) / (baseline_trades['pnl'].sum() / baseline_trades['entry_cost'].sum())
    if capital_efficiency > 1:
        print(f"   ✅ Better capital efficiency: {(capital_efficiency - 1)*100:.1f}% improvement")
    else:
        print(f"   ❌ Worse capital efficiency: {(1 - capital_efficiency)*100:.1f}% decline")

print("=" * 70)

In [ ]:
# Step 5: Visual Comparison
if len(filtered_trades) > 0:
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Baseline cumulative P&L
    baseline_sorted = baseline_trades.sort_values('Date').copy()
    baseline_sorted['cumulative_pnl'] = baseline_sorted['pnl'].cumsum()
    
    axes[0].plot(baseline_sorted['Date'], baseline_sorted['cumulative_pnl'], 
                 linewidth=2, color='gray', label='Baseline (All Strikes)', alpha=0.7)
    axes[0].axhline(y=0, color='black', linestyle='--', alpha=0.3)
    axes[0].set_xlabel('Date')
    axes[0].set_ylabel('Cumulative P&L ($)')
    axes[0].set_title(f'Baseline: All Strikes\nFinal: ${baseline_sorted["cumulative_pnl"].iloc[-1]:.2f}')
    axes[0].grid(alpha=0.3)
    axes[0].tick_params(axis='x', rotation=45)
    axes[0].legend()
    
    # Filtered cumulative P&L
    filtered_sorted = filtered_trades.sort_values('Date').copy()
    filtered_sorted['cumulative_pnl'] = filtered_sorted['pnl'].cumsum()
    
    axes[1].plot(filtered_sorted['Date'], filtered_sorted['cumulative_pnl'], 
                 linewidth=2, color='blue', label='±5% Filter')
    
    if 'double_trades' in locals() and len(double_trades) > 0:
        double_sorted = double_trades.sort_values('Date').copy()
        double_sorted['cumulative_pnl'] = double_sorted['pnl'].cumsum()
        axes[1].plot(double_sorted['Date'], double_sorted['cumulative_pnl'], 
                     linewidth=2, color='green', label='±5% + Bottom 5 Excluded', alpha=0.8)
    
    axes[1].axhline(y=0, color='black', linestyle='--', alpha=0.3)
    axes[1].set_xlabel('Date')
    axes[1].set_ylabel('Cumulative P&L ($)')
    axes[1].set_title(f'Filtered Strategies\nFinal: ${filtered_sorted["cumulative_pnl"].iloc[-1]:.2f}')
    axes[1].grid(alpha=0.3)
    axes[1].tick_params(axis='x', rotation=45)
    axes[1].legend()
    
    plt.tight_layout()
    plt.show()
    
    improvement = filtered_sorted['cumulative_pnl'].iloc[-1] - baseline_sorted['cumulative_pnl'].iloc[-1]
    print(f"\n📈 Strike Filtering Impact: ${improvement:+.2f}")

## 15. Strike Range Filtering Analysis - ±10%

**Purpose:** Test ±10% strike filter (less restrictive than ±5%) for comparison.

**Comparison:**
1. Baseline (All strikes, all tickers)
2. Conservative + ±10% filter
3. Conservative + ±10% filter + Bottom 5 tickers excluded

**Goal:** Determine if ±10% performs better, worse, or similar to ±5% filtering.

In [ ]:
# Step 1: Create ±10% filtered dataset
print("Creating ±10% strike filter...\n")

# Calculate distance from Exp Value as percentage (reuse if exists)
if 'distance_pct' not in data.columns:
    data['distance_pct'] = abs(data['Strike Price'] - data['Exp Value']) / data['Exp Value']

# Filter to ±10%
filtered_10pct = data[data['distance_pct'] <= 0.10].copy()

print("📊 Strike Filtering Results (±10%):")
print("=" * 70)
print(f"Original data: {len(data):,} contracts")
print(f"Filtered (±10%): {len(filtered_10pct):,} contracts")
print(f"Reduction: {(1 - len(filtered_10pct)/len(data)):.1%}")
print(f"\nOriginal tickers: {data['Ticker'].nunique()}")
print(f"Filtered tickers: {filtered_10pct['Ticker'].nunique()}")
print(f"Date range: {filtered_10pct['Date'].min().date()} to {filtered_10pct['Date'].max().date()}")
print("=" * 70)

In [ ]:
# Step 2: Run Conservative strategy on ±10% filtered data
print("\nRunning Conservative strategy (RSI 14, 25/75) on ±10% filtered data...\n")

filtered_10_results = backtest_multi_strike(filtered_10pct, rsi_period=14, oversold=25, overbought=75)
filtered_10_trades = filtered_10_results[filtered_10_results['signal'] != 0].copy()

if len(filtered_10_trades) == 0:
    print("WARNING: No trades generated!")
else:
    wins_10 = filtered_10_trades[filtered_10_trades['pnl'] > 0]
    losses_10 = filtered_10_trades[filtered_10_trades['pnl'] < 0]
    
    print("=" * 70)
    print("📊 FILTERED RESULTS (Conservative + ±10% Strike Filter)")
    print("=" * 70)
    print(f"Total Trades:           {len(filtered_10_trades):,}")
    print(f"Winning Trades:         {len(wins_10):,}")
    print(f"Losing Trades:          {len(losses_10):,}")
    print(f"Win Rate:               {len(wins_10)/len(filtered_10_trades):.2%}")
    print(f"\nTotal P&L:              ${filtered_10_trades['pnl'].sum():.2f}")
    print(f"Average Win:            ${wins_10['pnl'].mean():.2f}" if len(wins_10) > 0 else "N/A")
    print(f"Average Loss:           ${losses_10['pnl'].mean():.2f}" if len(losses_10) > 0 else "N/A")
    print(f"Avg Trade P&L:          ${filtered_10_trades['pnl'].mean():.2f}")
    print(f"\nTotal Capital Used:     ${filtered_10_trades['entry_cost'].sum():.2f}")
    print(f"Total Return:           {(filtered_10_trades['pnl'].sum() / filtered_10_trades['entry_cost'].sum() * 100):.2f}%")
    print(f"Sharpe Ratio:           {(filtered_10_trades['pnl'].mean() / filtered_10_trades['pnl'].std()) * np.sqrt(252):.2f}" if len(filtered_10_trades) > 1 and filtered_10_trades['pnl'].std() > 0 else "N/A")
    print("=" * 70)

In [ ]:
# Step 3: Filter BOTH by ±10% AND exclude bottom 5 tickers
if 'bottom_5' in locals() and 'filtered_data' in locals():
    print("\nApplying BOTH ±10% filter AND bottom 5 exclusion...\n")
    
    # Start with filtered_data from Section 13 (bottom 5 excluded)
    double_filtered_10 = filtered_data[filtered_data['Ticker'].isin(filtered_tickers)].copy()
    
    # Calculate distance and apply ±10% filter
    if 'distance_pct' not in double_filtered_10.columns:
        double_filtered_10['distance_pct'] = abs(double_filtered_10['Strike Price'] - double_filtered_10['Exp Value']) / double_filtered_10['Exp Value']
    double_filtered_10 = double_filtered_10[double_filtered_10['distance_pct'] <= 0.10].copy()
    
    print(f"Original data: {len(data):,} contracts")
    print(f"Bottom 5 excluded: {len(filtered_data):,} contracts")
    print(f"Both filters applied (±10%): {len(double_filtered_10):,} contracts")
    print(f"Total reduction: {(1 - len(double_filtered_10)/len(data)):.1%}\n")
    
    # Run backtest
    double_10_results = backtest_multi_strike(double_filtered_10, rsi_period=14, oversold=25, overbought=75)
    double_10_trades = double_10_results[double_10_results['signal'] != 0].copy()
    
    if len(double_10_trades) > 0:
        wins_dbl_10 = double_10_trades[double_10_trades['pnl'] > 0]
        losses_dbl_10 = double_10_trades[double_10_trades['pnl'] < 0]
        
        print("=" * 70)
        print("📊 DOUBLE FILTERED (±10% + Bottom 5 Excluded)")
        print("=" * 70)
        print(f"Total Trades:           {len(double_10_trades):,}")
        print(f"Winning Trades:         {len(wins_dbl_10):,}")
        print(f"Losing Trades:          {len(losses_dbl_10):,}")
        print(f"Win Rate:               {len(wins_dbl_10)/len(double_10_trades):.2%}")
        print(f"\nTotal P&L:              ${double_10_trades['pnl'].sum():.2f}")
        print(f"Average Win:            ${wins_dbl_10['pnl'].mean():.2f}" if len(wins_dbl_10) > 0 else "N/A")
        print(f"Average Loss:           ${losses_dbl_10['pnl'].mean():.2f}" if len(losses_dbl_10) > 0 else "N/A")
        print(f"Avg Trade P&L:          ${double_10_trades['pnl'].mean():.2f}")
        print(f"\nTotal Capital Used:     ${double_10_trades['entry_cost'].sum():.2f}")
        print(f"Total Return:           {(double_10_trades['pnl'].sum() / double_10_trades['entry_cost'].sum() * 100):.2f}%")
        print(f"Sharpe Ratio:           {(double_10_trades['pnl'].mean() / double_10_trades['pnl'].std()) * np.sqrt(252):.2f}" if len(double_10_trades) > 1 and double_10_trades['pnl'].std() > 0 else "N/A")
        print("=" * 70)
else:
    print("\n⚠️  Note: Run Section 13 first to enable double filtering (±10% + Bottom 5 exclusion)")

In [ ]:
# Step 4: Comprehensive Comparison (±10%)
print("\n" + "=" * 70)
print("📊 COMPREHENSIVE COMPARISON - Conservative Strategy (RSI 14, 25/75)")
print("=" * 70)

# Re-run baseline for comparison (or reuse from Section 14)
if 'baseline_trades' not in locals():
    baseline_results = backtest_multi_strike(data, rsi_period=14, oversold=25, overbought=75)
    baseline_trades = baseline_results[baseline_results['signal'] != 0].copy()

print(f"\n1️⃣  BASELINE (All Strikes, All Tickers):")
print(f"   Trades: {len(baseline_trades):,}")
print(f"   Win Rate: {(len(baseline_trades[baseline_trades['pnl'] > 0]) / len(baseline_trades)):.2%}")
print(f"   Total P&L: ${baseline_trades['pnl'].sum():.2f}")
print(f"   Return: {(baseline_trades['pnl'].sum() / baseline_trades['entry_cost'].sum() * 100):.2f}%")
print(f"   Capital Used: ${baseline_trades['entry_cost'].sum():.2f}")

if len(filtered_10_trades) > 0:
    print(f"\n2️⃣  WITH ±10% STRIKE FILTER:")
    print(f"   Trades: {len(filtered_10_trades):,} ({(len(filtered_10_trades)/len(baseline_trades) - 1)*100:+.1f}%)")
    print(f"   Win Rate: {len(wins_10)/len(filtered_10_trades):.2%} ({(len(wins_10)/len(filtered_10_trades) - len(baseline_trades[baseline_trades['pnl'] > 0])/len(baseline_trades))*100:+.1f}pp)")
    print(f"   Total P&L: ${filtered_10_trades['pnl'].sum():.2f} ({(filtered_10_trades['pnl'].sum() - baseline_trades['pnl'].sum()):+.2f})")
    print(f"   Return: {(filtered_10_trades['pnl'].sum() / filtered_10_trades['entry_cost'].sum() * 100):.2f}% ({(filtered_10_trades['pnl'].sum()/filtered_10_trades['entry_cost'].sum() - baseline_trades['pnl'].sum()/baseline_trades['entry_cost'].sum())*100:+.2f}pp)")
    print(f"   Capital Used: ${filtered_10_trades['entry_cost'].sum():.2f} ({(filtered_10_trades['entry_cost'].sum()/baseline_trades['entry_cost'].sum() - 1)*100:+.1f}%)")

if 'double_10_trades' in locals() and len(double_10_trades) > 0:
    print(f"\n3️⃣  WITH ±10% FILTER + BOTTOM 5 EXCLUDED:")
    print(f"   Trades: {len(double_10_trades):,} ({(len(double_10_trades)/len(baseline_trades) - 1)*100:+.1f}%)")
    print(f"   Win Rate: {len(wins_dbl_10)/len(double_10_trades):.2%} ({(len(wins_dbl_10)/len(double_10_trades) - len(baseline_trades[baseline_trades['pnl'] > 0])/len(baseline_trades))*100:+.1f}pp)")
    print(f"   Total P&L: ${double_10_trades['pnl'].sum():.2f} ({(double_10_trades['pnl'].sum() - baseline_trades['pnl'].sum()):+.2f})")
    print(f"   Return: {(double_10_trades['pnl'].sum() / double_10_trades['entry_cost'].sum() * 100):.2f}% ({(double_10_trades['pnl'].sum()/double_10_trades['entry_cost'].sum() - baseline_trades['pnl'].sum()/baseline_trades['entry_cost'].sum())*100:+.2f}pp)")
    print(f"   Capital Used: ${double_10_trades['entry_cost'].sum():.2f} ({(double_10_trades['entry_cost'].sum()/baseline_trades['entry_cost'].sum() - 1)*100:+.1f}%)")

print("\n" + "=" * 70)
print("\n💡 KEY INSIGHTS (±10% Filter):")
if len(filtered_10_trades) > 0:
    pnl_change = filtered_10_trades['pnl'].sum() - baseline_trades['pnl'].sum()
    if pnl_change > 0:
        print(f"   ✅ ±10% strike filtering IMPROVED P&L by ${pnl_change:.2f}")
    else:
        print(f"   ❌ ±10% strike filtering REDUCED P&L by ${abs(pnl_change):.2f}")
    
    capital_efficiency = (filtered_10_trades['pnl'].sum() / filtered_10_trades['entry_cost'].sum()) / (baseline_trades['pnl'].sum() / baseline_trades['entry_cost'].sum())
    if capital_efficiency > 1:
        print(f"   ✅ Better capital efficiency: {(capital_efficiency - 1)*100:.1f}% improvement")
    else:
        print(f"   ❌ Worse capital efficiency: {(1 - capital_efficiency)*100:.1f}% decline")

print("=" * 70)

In [ ]:
# Step 5: Side-by-Side Comparison of ±5% vs ±10%
print("\n" + "=" * 70)
print("🎯 FINAL COMPARISON: ±5% vs ±10% Strike Filters")
print("=" * 70)

if 'filtered_trades' in locals() and 'double_trades' in locals() and len(filtered_trades) > 0 and len(double_trades) > 0:
    print("\n📊 ±5% FILTER RESULTS:")
    print(f"   Strike Only:          {len(filtered_trades):,} trades, Win Rate: {len(filtered_trades[filtered_trades['pnl'] > 0])/len(filtered_trades):.2%}, P&L: ${filtered_trades['pnl'].sum():.2f}, Return: {(filtered_trades['pnl'].sum()/filtered_trades['entry_cost'].sum()*100):.2f}%")
    print(f"   + Bottom 5 Excluded:  {len(double_trades):,} trades, Win Rate: {len(double_trades[double_trades['pnl'] > 0])/len(double_trades):.2%}, P&L: ${double_trades['pnl'].sum():.2f}, Return: {(double_trades['pnl'].sum()/double_trades['entry_cost'].sum()*100):.2f}%")

if len(filtered_10_trades) > 0 and 'double_10_trades' in locals() and len(double_10_trades) > 0:
    print("\n📊 ±10% FILTER RESULTS:")
    print(f"   Strike Only:          {len(filtered_10_trades):,} trades, Win Rate: {len(wins_10)/len(filtered_10_trades):.2%}, P&L: ${filtered_10_trades['pnl'].sum():.2f}, Return: {(filtered_10_trades['pnl'].sum()/filtered_10_trades['entry_cost'].sum()*100):.2f}%")
    print(f"   + Bottom 5 Excluded:  {len(double_10_trades):,} trades, Win Rate: {len(wins_dbl_10)/len(double_10_trades):.2%}, P&L: ${double_10_trades['pnl'].sum():.2f}, Return: {(double_10_trades['pnl'].sum()/double_10_trades['entry_cost'].sum()*100):.2f}%")

print("\n" + "=" * 70)
if 'double_trades' in locals() and 'double_10_trades' in locals() and len(double_trades) > 0 and len(double_10_trades) > 0:
    winner_5 = double_trades['pnl'].sum()
    winner_10 = double_10_trades['pnl'].sum()
    if winner_5 > winner_10:
        print(f"\n🏆 WINNER: ±5% + Bottom 5 Excluded (${winner_5:.2f} vs ${winner_10:.2f})")
        print(f"   Advantage: ${winner_5 - winner_10:.2f} ({((winner_5/winner_10 - 1)*100):.1f}% better)")
    else:
        print(f"\n🏆 WINNER: ±10% + Bottom 5 Excluded (${winner_10:.2f} vs ${winner_5:.2f})")
        print(f"   Advantage: ${winner_10 - winner_5:.2f} ({((winner_10/winner_5 - 1)*100):.1f}% better)")

print("=" * 70)

## 16. Position Sizing Simulation

**Purpose:** Simulate realistic trading with position sizing constraints.

**Rules:**
- Starting capital: $500
- Max daily allocation: 10% of available capital
- Track capital growth/decline over time
- Use optimal strategy: Conservative (RSI 14, 25/75) + ±10% filter + Bottom 5 excluded

**Goal:** Understand capital requirements and realistic returns with position sizing limits.

In [ ]:
def simulate_position_sizing(trades_df: pd.DataFrame, starting_capital: float = 500.0, 
                             daily_allocation_pct: float = 0.10) -> pd.DataFrame:
    """
    Simulate trading with position sizing constraints.
    
    Args:
        trades_df: DataFrame with trades (must have 'Date', 'entry_cost', 'pnl' columns)
        starting_capital: Initial capital
        daily_allocation_pct: Max % of capital to allocate per day
    
    Returns:
        DataFrame with daily capital and trade execution details
    """
    # Sort by date
    trades = trades_df.sort_values('Date').copy()
    
    # Initialize tracking
    capital = starting_capital
    daily_results = []
    
    # Group trades by date
    for date, day_trades in trades.groupby('Date'):
        # Calculate total capital needed for all trades this day (without constraints)
        total_needed = day_trades['entry_cost'].sum()
        
        # Calculate max we can allocate today
        max_allocation = capital * daily_allocation_pct
        
        # Determine how many trades we can actually take
        if total_needed <= max_allocation:
            # Can take all trades
            executed_trades = day_trades.copy()
            actual_allocation = total_needed
        else:
            # Must select subset of trades
            # Strategy: Take trades proportionally until we hit limit
            executed_trades = []
            remaining_allocation = max_allocation
            
            for idx, trade in day_trades.iterrows():
                if trade['entry_cost'] <= remaining_allocation:
                    executed_trades.append(trade)
                    remaining_allocation -= trade['entry_cost']
                else:
                    # Can't afford this trade
                    break
            
            if executed_trades:
                executed_trades = pd.DataFrame(executed_trades)
                actual_allocation = executed_trades['entry_cost'].sum()
            else:
                executed_trades = pd.DataFrame()
                actual_allocation = 0
        
        # Calculate P&L for executed trades
        if len(executed_trades) > 0:
            daily_pnl = executed_trades['pnl'].sum()
            trades_executed = len(executed_trades)
        else:
            daily_pnl = 0
            trades_executed = 0
        
        # Update capital
        capital += daily_pnl
        
        # Record results
        daily_results.append({
            'Date': date,
            'starting_capital': capital - daily_pnl,
            'max_allocation': max_allocation,
            'actual_allocation': actual_allocation,
            'total_available_trades': len(day_trades),
            'trades_executed': trades_executed,
            'trades_skipped': len(day_trades) - trades_executed,
            'daily_pnl': daily_pnl,
            'ending_capital': capital,
            'capital_utilization_pct': (actual_allocation / max_allocation * 100) if max_allocation > 0 else 0
        })
    
    return pd.DataFrame(daily_results)

print("✓ Position sizing simulation function ready")

In [ ]:
# Run position sizing simulation on optimal strategy
print("Running position sizing simulation...")
print("Starting Capital: $500")
print("Daily Allocation Limit: 10% of available capital")
print("Strategy: Conservative (RSI 14, 25/75) + ±10% Strike Filter + Bottom 5 Excluded\n")

if 'double_10_trades' not in locals() or len(double_10_trades) == 0:
    print("⚠️  Warning: Optimal strategy trades not found. Run Section 15 first!")
else:
    # Run simulation
    sim_results = simulate_position_sizing(double_10_trades, starting_capital=500, daily_allocation_pct=0.10)
    
    print("=" * 70)
    print("📊 POSITION SIZING SIMULATION RESULTS")
    print("=" * 70)
    print(f"\nStarting Capital:       ${sim_results['starting_capital'].iloc[0]:.2f}")
    print(f"Ending Capital:         ${sim_results['ending_capital'].iloc[-1]:.2f}")
    print(f"Total P&L:              ${(sim_results['ending_capital'].iloc[-1] - sim_results['starting_capital'].iloc[0]):.2f}")
    print(f"Total Return:           {((sim_results['ending_capital'].iloc[-1] / sim_results['starting_capital'].iloc[0] - 1) * 100):.2f}%")
    print(f"\nTrades Available:       {sim_results['total_available_trades'].sum():,}")
    print(f"Trades Executed:        {sim_results['trades_executed'].sum():,}")
    print(f"Trades Skipped:         {sim_results['trades_skipped'].sum():,} (due to capital constraints)")
    print(f"Execution Rate:         {(sim_results['trades_executed'].sum() / sim_results['total_available_trades'].sum() * 100):.1f}%")
    print(f"\nAvg Daily Allocation:   ${sim_results['actual_allocation'].mean():.2f}")
    print(f"Max Daily Allocation:   ${sim_results['actual_allocation'].max():.2f}")
    print(f"Avg Utilization:        {sim_results['capital_utilization_pct'].mean():.1f}%")
    print(f"\nBest Day:               ${sim_results['daily_pnl'].max():.2f}")
    print(f"Worst Day:              ${sim_results['daily_pnl'].min():.2f}")
    print(f"Avg Daily P&L:          ${sim_results['daily_pnl'].mean():.2f}")
    print(f"Winning Days:           {len(sim_results[sim_results['daily_pnl'] > 0])} ({(len(sim_results[sim_results['daily_pnl'] > 0]) / len(sim_results) * 100):.1f}%)")
    print("=" * 70)
    
    print("\nDaily Breakdown:")
    display(sim_results[['Date', 'starting_capital', 'max_allocation', 'trades_executed', 
                         'trades_skipped', 'daily_pnl', 'ending_capital']].round(2))

In [ ]:
# Visualization: Capital Growth with Position Sizing
if 'sim_results' in locals():
    fig, axes = plt.subplots(2, 2, figsize=(16, 10))
    
    # Capital growth over time
    axes[0, 0].plot(sim_results['Date'], sim_results['ending_capital'], linewidth=2, marker='o')
    axes[0, 0].axhline(y=500, color='red', linestyle='--', alpha=0.5, label='Starting Capital')
    axes[0, 0].fill_between(sim_results['Date'], 500, sim_results['ending_capital'],
                            where=(sim_results['ending_capital'] >= 500), alpha=0.3, color='green')
    axes[0, 0].fill_between(sim_results['Date'], 500, sim_results['ending_capital'],
                            where=(sim_results['ending_capital'] < 500), alpha=0.3, color='red')
    axes[0, 0].set_xlabel('Date')
    axes[0, 0].set_ylabel('Capital ($)')
    axes[0, 0].set_title(f'Capital Growth Over Time\nFinal: ${sim_results["ending_capital"].iloc[-1]:.2f}')
    axes[0, 0].grid(alpha=0.3)
    axes[0, 0].tick_params(axis='x', rotation=45)
    axes[0, 0].legend()
    
    # Daily P&L
    colors = ['green' if x > 0 else 'red' for x in sim_results['daily_pnl']]
    axes[0, 1].bar(sim_results['Date'], sim_results['daily_pnl'], color=colors, alpha=0.6)
    axes[0, 1].axhline(y=0, color='black', linestyle='-', alpha=0.3)
    axes[0, 1].set_xlabel('Date')
    axes[0, 1].set_ylabel('Daily P&L ($)')
    axes[0, 1].set_title('Daily P&L with Position Sizing')
    axes[0, 1].grid(alpha=0.3)
    axes[0, 1].tick_params(axis='x', rotation=45)
    
    # Trade execution rate
    axes[1, 0].bar(sim_results['Date'], sim_results['trades_executed'], label='Executed', alpha=0.7)
    axes[1, 0].bar(sim_results['Date'], sim_results['trades_skipped'], 
                   bottom=sim_results['trades_executed'], label='Skipped (Capital Limit)', alpha=0.7, color='red')
    axes[1, 0].set_xlabel('Date')
    axes[1, 0].set_ylabel('Number of Trades')
    axes[1, 0].set_title('Trade Execution: Actual vs Skipped')
    axes[1, 0].legend()
    axes[1, 0].grid(alpha=0.3)
    axes[1, 0].tick_params(axis='x', rotation=45)
    
    # Capital utilization
    axes[1, 1].bar(sim_results['Date'], sim_results['capital_utilization_pct'], alpha=0.7)
    axes[1, 1].axhline(y=100, color='red', linestyle='--', alpha=0.5, label='100% Utilization')
    axes[1, 1].set_xlabel('Date')
    axes[1, 1].set_ylabel('Utilization (%)')
    axes[1, 1].set_title('Daily Capital Utilization (% of 10% Limit Used)')
    axes[1, 1].legend()
    axes[1, 1].grid(alpha=0.3)
    axes[1, 1].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    # Summary comparison
    print("\n" + "=" * 70)
    print("🎯 COMPARISON: Full Strategy vs Position-Sized")
    print("=" * 70)
    print(f"\nFULL STRATEGY (no capital limits):")
    print(f"  Total P&L:          ${double_10_trades['pnl'].sum():.2f}")
    print(f"  Capital Required:   ${double_10_trades['entry_cost'].sum():.2f}")
    print(f"  Return:             {(double_10_trades['pnl'].sum() / double_10_trades['entry_cost'].sum() * 100):.2f}%")
    print(f"  All Trades:         {len(double_10_trades):,}")
    
    print(f"\nPOSITION-SIZED ($500 start, 10% daily):")
    final_capital = sim_results['ending_capital'].iloc[-1]
    starting = sim_results['starting_capital'].iloc[0]
    print(f"  Total P&L:          ${(final_capital - starting):.2f}")
    print(f"  Capital Required:   ${starting:.2f} (starting)")
    print(f"  Return:             {((final_capital / starting - 1) * 100):.2f}%")
    print(f"  Trades Executed:    {sim_results['trades_executed'].sum():,} ({(sim_results['trades_executed'].sum() / len(double_10_trades) * 100):.1f}% of available)")
    print("=" * 70)